In [ ]:

##  better software principles w/ dataset class,
    # what does a dataset have? normalizing functions, 

# how we're gonna do synthetic data
    ## do the convolution thing once we have one band's carma fit to get others, then manually add lags
    
## could test the effects of length / num points on anomaly detection w/ syntehtic data, anomalous fits
# for dataset balancing, its probably more about number of observed points
# for training, its more about light cures being in the same time ranges 


## 12/27
    ## mcmc sample errors on the fly during training, error bar code 

# weird decisions to make (for talkinig purposes)
    # normlaize across training or individually? 
    # how to 'chop' by lenght or num points, min points?
    # injecting domain-specific knowledge
        # >1 mag err, keep obejcts between 13-20 mag, objects w/ certain intrinsitc var
    # formatting, setting masks
    
    # hyper-parameter things
    
    # how to use error bars on input data? 
        
        
    # lr scheduling? 
    # visualize kl annealing? wait_until_kl_inc is a hyperparam! 

In [94]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from dataset import DataSet
from eztao.carma import DRW_term, DHO_term
from eztao.ts import gpSimRand
import os
import model
import torch.optim as optim
import logging
from eztao.carma import DRW_term, DHO_term
from eztao.ts import gpSimRand, gpSimFull


def get_synth_data(folder, seed = 0, batch_size=8, frac=0.5, kernel='drw', duration=730, n=180):
    """
    
    This function creates and loads a synthetic dataset relative to a given kernel (drw or dho), 
    distributing the kernel params relative to a real dataset, in base_folder 
    
    parameters:
        seed               (int)      --> random seed, this matters to keep the shuffles consistent
        batch_size         (int)      --> for the network, usually a multiple of 2
        kernel             (str)      --> 'drw' (dampled random walk), a carma(1) process; 'dho' (damped harmonic oscillator), a carma(2,1) process
        
    drw_kernel def --> 'tau' is decorelation timescale, 'amp' is the amplitude
    dho_kernel def --> a1 = 2 * Xi * w0
                       a2 = w0 ** 2
                       b0 = sigma
                       b1 = tau * b0
                       wherein Xi is the damping ratio, w0 is the natural oscillation frequency, sigma is the amplitude
                       of the short term perturbing white noise, tau is the characteristic timescale of the perturbation process
    returns:
        a dictionary of torch dataloaders with data formatted as necessary for network training 
        , as well as the dimension and union of all the time points
    """
    # which bands do you want to simulate?!?
    
    np.random.seed(seed)
    if not os.path.isdir(folder):
        raise Exception(f"{folder} is not a directory")
        
    lcs = DataSet(name=folder, min_length=50, sep=',', start_col=1)
    band_folders = os.listdir(folder)
    for band_folder in band_folders:
        band = band_folder.lower()
        lcs.add_band(band, os.path.join(folder, band_folder))
        
    lcs.filter()         
    lcs.prune_outliers()
    lcs.set_carma_fits()
    lcs.set_snr()
    if kernel == 'drw':
        synth_lcs = []
        for i, params in enumerate(lcs.drw_fits):
            if np.isnan(params[0]):
                continue
            DRW_kernel = DRW_term(*np.log(params))
            # kernel, snr, duration (days), n 
            lc = np.array(gpSimRand(DRW_kernel,lcs.snr[i,0], duration, n)).transpose(1,0)[np.newaxis,np.newaxis]
            synth_lcs.append(lc)
        synth_lcs = np.concatenate(synth_lcs, axis=0)
    ##################### dho #################################
    else:
        synth_lcs = []
        for i, params in enumerate(lcs.dho_fits):
            if np.isnan(params[0]):
                print('nannnnnnn')
                continue
            DHO_kernel = DHO_term(*np.log(params))
            # kernel, snr, duration (days), n 
            lc = np.array(gpSimRand(DHO_kernel,lcs.snr[i,0], duration, n)).transpose(1,0)[np.newaxis,np.newaxis]
            synth_lcs.append(lc)
        synth_lcs = np.concatenate(synth_lcs, axis=0)
    
    print(synth_lcs.shape)
    union_tp = np.unique(synth_lcs[:,:,:,0].flatten()).astype('float32')
    union_tp = np.arange(0, np.ptp(union_tp), step=0.5)
    
    np.random.shuffle(synth_lcs)
    splindex = int(np.floor(.8*len(synth_lcs)))
    training, valid, test = np.split(synth_lcs, [splindex, splindex + int(np.floor(.1*len(synth_lcs)))])# shuffle?
    train_loader = torch.utils.data.DataLoader(training, batch_size=batch_size)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size)
    data_objects = {
        "train_loader": train_loader,
        "test_loader": test_loader,
        "valid_loader": valid_loader,
        'union_tp': union_tp,
        "input_dim": 1,
    }
    return data_objects


In [90]:
%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
%autoreload

In [92]:
get_synth_data('test_data', kernel='drw')

validated 55 files out of 55 for band='g'
0

/opt/homebrew/Caskroom/miniforge/base/envs/hetvae/lib/python3.10/site-packages/eztao/ts/carma_fit.py:423: RuntimeWarning: divide by zero encountered in log
  (np.log(min_dt / 5), np.log(t[-1])),
/opt/homebrew/Caskroom/miniforge/base/envs/hetvae/lib/python3.10/site-packages/eztao/ts/carma_fit.py:434: RuntimeWarning: divide by zero encountered in log
  log_tau_range = [np.log(min_dt / 5), np.log(t[-1] / 10)]


123456789101112131415161718192021222324252627282930313233343536373839[nan, nan]
[nan, nan]
[0.20783707 6.22633928]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
[1.61934227e-01 4.55184695e+02]
[nan, nan]
[nan, nan]
[nan, nan]
[nan, nan]
(2, 1, 180, 3)


{'train_loader': <torch.utils.data.dataloader.DataLoader at 0x153d9c910>,
 'test_loader': <torch.utils.data.dataloader.DataLoader at 0x2880644c0>,
 'valid_loader': <torch.utils.data.dataloader.DataLoader at 0x288065360>,
 'union_tp': array([0.000e+00, 5.000e-01, 1.000e+00, ..., 5.390e+02, 5.395e+02,
        5.400e+02]),
 'input_dim': 1}

In [ ]:
# check out previews
    
